# Consume Crack classifier as gRPC service

In [1]:
#!pip install tensorflow-serving-api==1.12

In [2]:
HOSTNAME="localhost"
MODEL="febrero-cpu-friendly_weights"
INPUT_IMAGE_FILENAME = "../../media/no_crack.jpg"
IMAGE_WIDTH=64
IMAGE_HEIGHT=64

In [3]:
def load_local_image(img_filename, w,h):
    img = cv2.imread(img_filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # fix color order (depends on OpenCV version)
    return cv2.resize(img,(int(w),int(h)))

## gRPC API

In [4]:
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
import grpc
from PIL import Image
import numpy as np
import requests
import cv2

In [29]:
def load_input_tensor(input_image):
    image_np = load_local_image(input_image,IMAGE_WIDTH,IMAGE_HEIGHT)
    image_np_expanded = np.expand_dims(image_np, axis=0).astype(np.float)
    tensor = tf.contrib.util.make_tensor_proto(image_np_expanded, dtype=tf.float32,shape=[1,IMAGE_WIDTH, IMAGE_HEIGHT,3])
    return tensor

SERVICE_ENDPOINT = 'localhost:8500'
channel = grpc.insecure_channel(SERVICE_ENDPOINT)

# Send request
# See prediction_service.proto for gRPC request/response details.
request = predict_pb2.PredictRequest()
request.model_spec.name = MODEL
request.model_spec.signature_name = 'serving_default'
input_tensor = load_input_tensor(INPUT_IMAGE_FILENAME)
request.inputs['input_image'].CopyFrom(input_tensor)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
result = stub.Predict(request, 10.0)  # 10 secs timeout
prob_crack = result.outputs["dense_12/Softmax:0"].float_val[1]
prob_nocrack = result.outputs["dense_12/Softmax:0"].float_val[0]
print("p(crack)=",prob_crack, "p(no_crack)=",prob_nocrack)

p(crack)= 0.0 p(no_crack)= 1.0


## REST API

Check model status and Predict signature.

In [6]:
REST_API_ENDPOINT='http://%s:8501/v1/models/%s' % (HOSTNAME, MODEL)

r = requests.get(REST_API_ENDPOINT)
r.json()

{'model_version_status': [{'state': 'AVAILABLE',
   'status': {'error_code': 'OK', 'error_message': ''},
   'version': '1'}]}

In [7]:
r = requests.get(REST_API_ENDPOINT+"/metadata")
r.json()

{'metadata': {'signature_def': {'signature_def': {'serving_default': {'inputs': {'input_image': {'dtype': 'DT_FLOAT',
       'name': 'conv2d_11_input:0',
       'tensor_shape': {'dim': [{'name': '', 'size': '-1'},
         {'name': '', 'size': '64'},
         {'name': '', 'size': '64'},
         {'name': '', 'size': '3'}],
        'unknown_rank': False}}},
     'method_name': 'tensorflow/serving/predict',
     'outputs': {'dense_12/Softmax:0': {'dtype': 'DT_FLOAT',
       'name': 'dense_12/Softmax:0',
       'tensor_shape': {'dim': [{'name': '', 'size': '-1'},
         {'name': '', 'size': '2'}],
        'unknown_rank': False}}}}}}},
 'model_spec': {'name': 'febrero-cpu-friendly_weights',
  'signature_name': '',
  'version': '1'}}

Make a predicition.

In [8]:
img = load_local_image(INPUT_IMAGE_FILENAME,IMAGE_WIDTH,IMAGE_HEIGHT)
r = requests.post(REST_API_ENDPOINT+':predict', json={"instances": [{'input_image': img.tolist()}]})
result = r.json()
print(result)

{'predictions': [[1.0, 0.0]]}
